In [12]:
import numpy as np
import librosa
import IPython.display as ipd
import random
import scipy.stats as st
import scipy.special as ss

import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
y, sr = librosa.load('audio/NickDrake_8k.wav')
z, _= librosa.load('audio/Loureed_8k.wav')
w, _= librosa.load('audio/tom1_8k.wav')
T = len(y)
S = np.array([z,w,y])
theta = np.array([np.pi/2, np.pi-np.pi/8, np.pi/8])/2
A = np.array([np.sin(theta), np.cos(theta)])
m , n = A.shape
sigma = 0.01
N = st.norm.rvs(loc = 0, scale = sigma, size = (m,T) )
X  = np.dot(A,S) + N

In [14]:
from scipy.fftpack import dct, idct

CX = dct(X, norm = 'ortho')
CX.shape

(2, 180634)

In [5]:
A

array([[0.70710678, 0.98078528, 0.19509032],
       [0.70710678, 0.19509032, 0.98078528],
       [0.19509032, 0.70710678, 0.98078528]])

In [15]:
#%%% Priors %%%

#% Inverse-Gamma prior on sigma_sqrd
shape_sigma_sqrd_prior = 1
scale_sigma_sqrd_prior = 1

#% Gamma prior on lambda    
shape_lambda_prior = np.zeros((n,1))
scale_lambda_prior = np.zeros((n,1))

#% Exponential prior on alpha
scale_alpha_prior = np.ones((n,1))

shape_v_prior = np.ones((n,1))
scale_v_prior = np.ones((T,1))     #vamos gerar uma matriz n*T, onde cada linha representa V_i,k 


#%% Gaussian prior on the columns of A
#%sigma_A = 100 * ones(n,1)

#%%% Initializations %%%
#cur_A = np.array([[1, 1, 1],[0, 0, 0]])

#% Variance of the noise initialisation
#%cur_sigma_sqrd=(sigma)^2
cur_sigma_sqrd=0.1

#% Sources scale parameter initialisation
cur_lambda=1*np.ones((n,1))
#%cur_beta = betaw

cur_alpha = 0.5*np.ones((n,1))

cur_v = 0.5*np.ones((n,T))

#% Sources coefficients initialisation  S
cur_Source_C= 0.5*np.ones((n,T))


SkXsk = np.ones((n,n))
Sigma_sk = np.ones((T,n,n))
Mu_sk = np.ones((T,n))
              
              
mu_A = np.ones((m,n))
cur_A = np.ones((m,n))
sigma_A =np.ones((n,n))
aux2_v = np.ones((n,T))

In [16]:
def p(x,i_n):
    return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
    
def q(x,x2):
    return st.gamma.logpdf(x,a=1,loc = x2)
def MH(x,x_star,p,q,i_n):
    return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])

In [17]:
import time

In [ ]:
# separa os list comprehent alocando memoria antecipadamente

usar numba e jit para tentar otimizar os códigos. Passar as variáveis em função para amostrar  

fazer o teste com três observações, problema com solução única

In [18]:
from numba import njit, jit, typeof

In [8]:
@njit
def aux(cur_sigma_sqrd,A):
    return (1/cur_sigma_sqrd)*np.dot(A.T,A)

In [11]:
aux(cur_sigma_sqrd,2*A)

array([[41.52240932, 38.77677201, 40.91245306],
       [38.77677201, 59.99999993, 43.04813414],
       [40.91245306, 43.04813414, 78.47759062]])

In [21]:
start_time = time.time()
n_iter = 100
iter_MH = 2
for i in range(n_iter):

        @njit
        def aux():
            return (1/cur_sigma_sqrd)*np.dot(A.T,A)

        @njit
        def aux2():
            aux_2 = np.zeros((T,n))
            for ii in np.arange(T):
                aux_2[ii,:] = np.dot(A.T,CX[:,ii])
            return aux_2

        @njit
        def sigma_sk():
            Sigma_sk = np.zeros((T,n,n))
            aux_1 = aux()
            for ii in np.arange(T):
                z = aux_1 + np.diag(1/cur_v[:,ii])
                z = np.linalg.pinv(z)
                for jj in np.arange(n):
                    for kk in np.arange(n):
                        Sigma_sk[ii,jj,kk] = z[jj,kk]
            return Sigma_sk

        @njit
        def mu_sk():
            intern = np.zeros((T,n))
            for ii in np.arange(T):
                intern[ii] = (1/cur_sigma_sqrd)*np.dot(Sigma_sk[ii],aux_2[ii])
            return intern

        @njit
        def cur_Source(cur_Source):
            for ii in np.arange(T):
                try:
                    cur_Source[:,ii] = multivariate_normal(Mu_sk[ii],Sigma_sk[ii])
                except:
                    pass
            return cur_Source

        @njit
        def sksk():
            sks = np.zeros((n,n))
            for ii in np.arange(T):
                sks += np.outer(cur_Source_C[:,ii],cur_Source_C[:,ii].T)
            return np.linalg.inv(sks)

        @njit
        def beta_sig():
            intern = np.zeros((m,T,n))
            for ii in np.arange(m):
                for kk in np.arange(T):
                    intern[ii,kk] = CX[ii,kk]*cur_Source_C[:,kk]
            intern2 = np.zeros((m,n))

            for ii in np.arange(m):
                intern2[ii] = np.sum(intern[ii],axis = 0 ) 

            soma = 0
            for ii in np.arange(m):

                soma += np.dot(intern2[ii],np.dot(SkXsk,intern2[ii].reshape(-1,1)))[0]

            return 2/(np.sum(np.power(CX,2)) - soma), intern2    # intern2 = X_times_s

        @njit
        def sig_mu_A():
            intern = np.zeros((m,n))
            for ii in np.arange(X_times_s.shape[0]):
                intern[ii] = (np.dot(SkXsk,X_times_s[ii].reshape(-1,1))).reshape(1,-1)

            return cur_sigma_sqrd*SkXsk, intern

        @njit
        def cur_V():
            cur_v = np.zeros((n,T))
            aux2_v = np.zeros((n,T))
            shape_v_prior = (cur_alpha + 1)/2
            aux_v = np.multiply(cur_alpha,cur_lambda)
            aux2_v[::] = np.power(cur_Source_C,2)
            for ii in np.arange(n):
                for kk in np.arange(T):
                    cur_v[ii,kk] = 1/np.random.gamma(shape = shape_v_prior[ii][0], scale = 2/(aux_v[ii] + aux2_v[ii][kk])[0])
            return cur_v



        print(i)
  
        #amostrando s^till
        #aux_ =  aux()

        Sigma_sk[::] = sigma_sk()
        aux_2 = aux2()
        Mu_sk[::] = mu_sk()
        cur_Source_C[:,:] = cur_Source(cur_Source_C)       

        
        #amostrando sigma^2
        alpha_sigma = (T-n)*m/2
        try:
            SkXsk[::] = sksk() #inverse matrix sum sk*skT

        
        except:
            pass
        
        beta_sigma, X_times_s = beta_sig()
        try:
            cur_sigma_sqrd= st.invgamma.rvs(a = alpha_sigma, scale = beta_sigma)

        except:
            pass
        
        
        #amostrando A
        sigma_A[::], mu_A[::] = sig_mu_A()
        #print('mu_A',mu_A[::])
        cur_A[::] = np.array([st.multivariate_normal.rvs(mean = k, cov = sigma_A ) for k in mu_A])
        #print('cur_A',cur_A)

        
        #amostrando V
        cur_v[::] = cur_V()

        #amostrando lambda
        R_i = (np.sum(1/cur_v, axis= 1))
        P_i = (np.prod(cur_v, axis= 1))
        shape_lambda_prior[::] = cur_alpha*T/2 
        scale_lambda_prior[::] = 2/np.multiply(R_i,cur_alpha.T).T
        cur_lambda[::] = st.gamma.rvs(a = shape_lambda_prior, scale = scale_lambda_prior)



        #amostrando alpha
        w = np.zeros(iter_MH+1)
        for j in range(n):
            for ii in range(iter_MH):
                u = st.uniform.rvs()
                x_star = st.gamma.rvs(a = 1,loc = w[ii])
            if u < MH(w[ii],x_star,p,q,j):
                w[ii+1] = x_star
            else:
                w[ii+1] = w[ii]

            cur_alpha[j] = w[-1]
            
    
print("--- %s seconds ---" % (time.time() - start_time))
       

0


<ipython-input-21-d42d840c5faf>:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (readonly array(float64, 2d, F), readonly array(float64, 1d, A))
  aux_2[ii,:] = np.dot(A.T,CX[:,ii])
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-16-261690351d82>:2: RuntimeWarning: divide by zero encountered in log
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:2: RuntimeWarning: invalid value encountered in multiply
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in less
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_st

1


<ipython-input-21-d42d840c5faf>:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (readonly array(float64, 2d, F), readonly array(float64, 1d, A))
  aux_2[ii,:] = np.dot(A.T,CX[:,ii])
<ipython-input-16-261690351d82>:2: RuntimeWarning: divide by zero encountered in log
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in add
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])
<ipython-input-16-261690351d82>:2: RuntimeWarning: invalid value encountered in multiply
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in less
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])


2


...

<ipython-input-21-d42d840c5faf>:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (readonly array(float64, 2d, F), readonly array(float64, 1d, A))
  aux_2[ii,:] = np.dot(A.T,CX[:,ii])
<ipython-input-16-261690351d82>:2: RuntimeWarning: divide by zero encountered in log
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in add
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])
<ipython-input-16-261690351d82>:2: RuntimeWarning: invalid value encountered in multiply
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in less
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])


99


<ipython-input-21-d42d840c5faf>:14: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (readonly array(float64, 2d, F), readonly array(float64, 1d, A))
  aux_2[ii,:] = np.dot(A.T,CX[:,ii])


--- 722.8490288257599 seconds ---


<ipython-input-16-261690351d82>:2: RuntimeWarning: divide by zero encountered in log
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in add
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])
<ipython-input-16-261690351d82>:2: RuntimeWarning: invalid value encountered in multiply
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))
<ipython-input-16-261690351d82>:7: RuntimeWarning: invalid value encountered in less
  return min([1,np.exp(p(x_star,i_n) + q(x,x_star) - p(x,i_n) - q(x_star,x))])


In [20]:
@jit(nopython=True)
def multivariate_normal(m,C):
    L = np.linalg.cholesky(C)
    X = np.random.randn(m.shape[0])
    return(L @ X + m)

In [17]:
@njit    #checar a scala se é lambda ou 1/lambda
def inv_gamma_n(alpha,scale): #aqui a gamma gerada pelo soma de exponenciais de parametro lambda, tem parametro n e 1/lambda
    soma = 0
    for i in np.arange(alpha):    
        soma += random.expovariate(scale)  #por isso tem scale aqui
    return soma

In [18]:
@njit
def inv_gamma_samp(shape,scale):
    u = random.random()
    amostra = inv_gamma_n(round(shape+1),scale)  #amostra candidata para inversa gamma 
    gamma_div_g = scale**(shape - round(shape+1))*((1/amostra)**(shape - round(shape+1)))

    if u<= gamma_div_g:
        return amostra
    return inv_gamma_samp(shape,scale)
    
    
    

In [ ]:
start_time = time.time()
n_iter = 100
iter_MH = 2
for i in range(n_iter):
        print(i)
  
        #amostrando s^till
        #aux_ =  aux()

        Sigma_sk[::] = sigma_sk()
        aux_2 = aux2()
        Mu_sk[::] = mu_sk()
        cur_Source_C[:,:] = cur_Source(cur_Source_C)       

        
        #amostrando sigma^2
        alpha_sigma = (T-n)*m/2
        try:
            SkXsk[::] = sksk() #inverse matrix sum sk*skT
            print("#"*40)
            print('entrou no sksk')
        
        except:
            pass
        
        beta_sigma, X_times_s = beta_sig()
        try:
            cur_sigma_sqrd= st.invgamma.rvs(a = alpha_sigma, scale = beta_sigma)
            print("#"*40)
            print('entrou no cur sigma inv gamma')
        except:
            pass
        
        
        #amostrando A
        sigma_A[::], mu_A[::] = sig_mu_A(X_times_s)
        #print('mu_A',mu_A[::])
        cur_A[::] = np.array([st.multivariate_normal.rvs(mean = k, cov = sigma_A ) for k in mu_A])
        #print('cur_A',cur_A)

        
        #amostrando V
        cur_v[::] = cur_V()

        #amostrando lambda
        R_i = (np.sum(1/cur_v, axis= 1))
        P_i = (np.prod(cur_v, axis= 1))
        shape_lambda_prior[::] = cur_alpha*T/2 
        scale_lambda_prior[::] = 2/np.multiply(R_i,cur_alpha.T).T
        cur_lambda[::] = st.gamma.rvs(a = shape_lambda_prior, scale = scale_lambda_prior)



        #amostrando alpha
        w = np.zeros(iter_MH+1)
        for j in range(n):
            for ii in range(iter_MH):
                u = st.uniform.rvs()
                x_star = st.gamma.rvs(a = 1,loc = w[ii])
            if u < MH(w[ii],x_star,p,q,j):
                w[ii+1] = x_star
            else:
                w[ii+1] = w[ii]

            cur_alpha[j] = w[-1]
            
    
print("--- %s seconds ---" % (time.time() - start_time))
   

0


In [132]:
SkXsk.shape

(3, 3)

In [22]:
cur_A - A

array([[ 1.50652669e-06,  3.74364225e-05, -1.14463790e-04],
       [ 8.36410921e-05, -1.09847079e-04,  7.26202922e-05]])

In [21]:
ipd.Audio(idct(cur_Source_C)[0], rate=sr)


In [20]:
ipd.Audio(idct(cur_Source_C)[1], rate=sr)


In [22]:
ipd.Audio(idct(cur_Source_C)[2], rate=sr)


In [23]:
ipd.Audio(idct(cur_Source_C)[0], rate=sr)


In [24]:
ipd.Audio(idct(cur_Source_C)[1], rate=sr)


In [25]:
ipd.Audio(idct(cur_Source_C)[2], rate=sr)


In [34]:
idct(cur_Source_C)

array([[ 1.14994374e+05, -3.83325276e+04,  2.29982329e+04, ...,
        -8.02269377e-01, -8.02251985e-01, -8.02260681e-01],
       [ 1.14994692e+05, -3.83322098e+04,  2.29985508e+04, ...,
        -4.84443865e-01, -4.84426473e-01, -4.84435169e-01],
       [ 1.14994965e+05, -3.83319358e+04,  2.29988247e+04, ...,
        -2.10541965e-01, -2.10524573e-01, -2.10533269e-01]])

In [11]:
np.linalg.inv(np.diag(cur_v[:,0]))

array([[18.5149899 ,  0.        ,  0.        ],
       [ 0.        ,  7.91501017,  0.        ],
       [ 0.        ,  0.        ,  0.35032513]])

In [12]:
np.diag(1/cur_v[:,0])

array([[18.5149899 ,  0.        ,  0.        ],
       [ 0.        ,  7.91501017,  0.        ],
       [ 0.        ,  0.        ,  0.35032513]])

In [55]:
x_star = st.gamma.rvs(a = 1,loc = w[ii])
x_star

cur_lambda
np.log(P_i)

<ipython-input-55-84dbe4b10b70>:5: RuntimeWarning: divide by zero encountered in log
  np.log(P_i)


array([-inf,  inf, -inf])

In [49]:
        w = np.ones(iter_MH+1)
        for j in range(n):
            for ii in range(iter_MH):
                u = st.uniform.rvs()
                x_star = st.gamma.rvs(a = 1,loc = w[ii])
            if u < MH(w[ii],x_star,p,q,j):
                w[ii+1] = x_star
            else:
                w[ii+1] = w[ii]

            cur_alpha[j] = w[-1]

<ipython-input-40-f7051a195c25>:2: RuntimeWarning: divide by zero encountered in log
  return ((np.log(P_i[i_n])*(-x/2 - 1) + (x*T/2)*np.log(x*cur_lambda[i_n]/2) - np.log(ss.gamma(x/2)))*T + (-x*cur_lambda[i_n]*R_i[i_n]/2))


TypeError: _parse_args() missing 1 required positional argument: 'a'

In [58]:
shape_lambda_prior[::] = cur_alpha*T/2 
scale_lambda_prior[::] = 2/np.multiply((np.sum(1/cur_v, axis= 1)),cur_alpha.T).T
cur_lambda[::] = st.gamma.rvs(a = shape_lambda_prior, scale = scale_lambda_prior)

In [53]:
2/np.multiply((np.sum(1/cur_v, axis= 1)),cur_alpha.T)

array([[7.87219289e-05, 7.88460829e-05, 7.88376617e-05]])

In [55]:
shape_lambda_prior[::] = cur_alpha*T/2 
scale_lambda_prior[::] = 2/np.multiply((np.sum(1/cur_v, axis= 1)),cur_alpha.T).T

In [34]:
shape_lambda_prior

array([[45158.5],
       [45158.5],
       [45158.5]])

In [18]:
shape_v_prior = (cur_alpha + 1)/2
aux_v = np.multiply(cur_alpha,cur_lambda)
aux2_v = np.power(cur_Source_C,2)
V_test = np.array([[st.invgamma.rvs(a = shape_v_prior[ii], scale = 2/(aux_v[ii] + aux2_v[ii][k])  ) for k in range(T)] for ii in range(n)])

array([ 3.48922661,  4.81414987,  1.19238353, ..., 52.13948101,
       56.43760375,  1.63100255])

In [152]:
cur_A = np.array([[1, 1, 1],[0, 0, 0]])

In [174]:
cur_A[:,:] = np.array([[1, 1, 1],[0, 0, 0]])

In [177]:
cur_A = np.array([st.multivariate_normal.rvs(mean = k, cov = sigma_A ) for k in mu_A])


In [178]:
cur_A

array([[0.30251286, 0.32252155, 0.17742992],
       [0.28323298, 0.17297806, 0.29774672]])

In [79]:
beta_sigma

0.00018953831564930468

In [53]:
#np.sum([np.outer(cur_Source_C[:,k],cur_Source_C[:,k].T) for k in range(T)],axis = 0)   # sum sk*skT 
X_times_s = np.array([[CX[k,j]*cur_Source_C[:,j] for j in range(T)] for k in range(m)])
X_times_s = np.array([np.sum(k,axis = 0 )for k in X_times_s ]) #soma de cada x_k_i*sk


array([[4.35334601e-05, 2.65624157e-05, 2.73029274e-05],
       [2.65624157e-05, 4.69567728e-05, 8.17756045e-06],
       [2.73029274e-05, 8.17756045e-06, 4.81241542e-05]])

In [81]:
[(np.matrix(SkXsk)*k.reshape(-1,1)).T for k in X_times_s ]

[matrix([[0.30013997, 0.32049972, 0.17787411]]),
 matrix([[0.28312749, 0.17349248, 0.29676119]])]

In [75]:
sum([float(k*np.matrix(SkXsk)*k.reshape(-1,1)) for k in X_times_s ])

5083.492056040932

In [55]:
CX.shape
X_times_s[0]

array([3437.80123426, 4714.4278433 ,  957.84096596])

In [49]:
[np.sum(k,axis = 0 )for k in X_times_s ]

[9076.301651636952, 8484.260360777864]

In [79]:
np.sum(np.power(CX,2)) - sum([float(k*np.matrix(SkXsk)*k.reshape(-1,1)) for k in X_times_s ])


15577.198244530475

In [195]:
np.linalg.inv(np.matrix([[1,-3/4,0],[-1/2,1,-1/2],[0,-3/4,1]]))

matrix([[2.5, 3. , 1.5],
        [2. , 4. , 2. ],
        [1.5, 3. , 2.5]])

In [193]:
np.matrix([[1,-3/4,1],[-1/2,1,-1/2],[0,-3/4,1]])

matrix([[ 1.  , -0.75,  1.  ],
        [-0.5 ,  1.  , -0.5 ],
        [ 0.  , -0.75,  1.  ]])